# Analyzing (RE)PPTIS simulations using an MSM approach
This notebook contains an example workflow that can be used for estimating the crossing probability and pathlengths of a (RE)PPTIS simulation.

## 1. Import the necessary functions

In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import matplotlib.pyplot as plt
from pprint import pprint    # to print the vars of the pathensemble object
import numpy as np
import os
import glob

# Reading
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights
from tistools import set_tau_distrib, set_tau_first_hit_M_distrib, cross_dist_distr, pathlength_distr
from tistools import collect_tau, collect_tau1, collect_tau2, collect_taum
from tistools import ACCFLAGS, REJFLAGS

# REPPTIS analysis
from tistools import get_lmr_masks, get_generation_mask, get_flag_mask, select_with_masks
from tistools import unwrap_by_weight, running_avg_local_probs, get_local_probs, get_global_probs_from_dict, get_global_probs_from_local

# MSM functions
from tistools import construct_M
from tistools import global_pcross_msm
from tistools import mfpt_to_first_last_state, mfpt_to_absorbing_states, construct_tau_vector
from tistools import create_labels_states, print_vector, print_all_tau

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Load the simulation data

In [26]:

# Set the working directory
indir = "/Users/an/Documents/0_mfpt/repptis1/"  

# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_2108"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_repptismazegap2708"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/i_star/simulations/RETIS_flat_br_noswap"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_abl/paper-msm-short-term/simulations-and-analysis/trypsin-benzamidine/infrepptisanalysis/tistools-mfptanalysis/"
indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS"

# zero_minus_one = True if lambda_-1 interface is set
# zero_minus_one = False if lambda_-1 interface is not set
zero_minus_one = False

inputfile = indir + "/repptis.rst"    # When using PyRETIS, the input file for REPPTIS simulations is a .rst file
# inputfile = indir + "/retis3.rst"

# Move to working directory
os.chdir(indir)
print(os.getcwd())

# Set the ensemble folders and print them
folders = glob.glob(indir + "/0[0-9][0-9]")
folders = sorted(folders)
print(folders)

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs
['/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/002', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/003', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/004', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/005', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/006', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/007', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/

In [27]:
# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i==1:
        pe.set_in_zero_plus(True)
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)

    
    # Read order parameters order.txt/order.npy into path ensemble object, or load from order.npy file.
    # Saving order parameter files allows to speed up this notebook.
    #### CHANGE HERE ####
    pe.set_orders(load=False, acc_only=True, save=True)        # for the 1st time you run this notebook for a certain simulation, this will store .npy files
    # pe.set_orders(load=True, acc_only=True, save=False)                  # for the next times, you can read npy files (save=True/False is not important)
    # pe.set_orders(load=False, acc_only=True, save=False)     # if saving doesn't work

################################################################################
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000
pathensemble info: 
{'cyclenumbers': array([    0,     1,     2, ..., 45028, 45029, 45030]),
 'flags': array(['ACC', 'REJ', 'ACC', ..., 'REJ', 'ACC', 'REJ'], dtype='<U3'),
 'generation': array(['sh', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': True,
 'in_zero_plus': False,
 'interfaces': [[1.0, 1.0, 1.0], ['l_[0]', 'l_[0]', 'l_[0]']],
 'lambmaxs': array([1.04186, 1.04186, 1.03438, ..., 1.03978, 1.02207, 1.02207]),
 'lambmins': array([0.94194, 0.94194, 0.94659, ..., 0.41419, 0.40912, 0.40912]),
 'lengths': array([ 7,  1, 11, ...,  1, 41,  1]),
 'lmrs': array(['RMR', 'RMR', 'RMR', ..., 'RMR', 'RMR', 'RMR'], dtype='<U3'),
 'name': '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000',
 'ncycle': 45031,


## 3. Regular (RE)PPTIS analysis using tistools

### Analyze the REPPTIS simulation.

In [28]:
# Analysis output is saved to the data dictionary.
data = {}
for i, pe in enumerate(pathensembles):
    print("doing pathensemble {}".format(i))
    if i == 0:
        data[i] = {}
        continue  #  [0-] is not used for Pcross calculations

    # Classify the paths according to their path type.
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
                (pe.lmrs == ptype).astype(int), pe.weights)
    
    # Running average analysis: ["running"]
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    for (ptype, p_loc) in zip(pathtypes, 
                              running_avg_local_probs(pathtype_cycles, 
                                                      pe.weights, tr = False)):
        data[i]["running"]["plocal"][ptype] = p_loc

    # Analysis using all data: ["full"]
    plocfull = get_local_probs(pe, tr=False)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]

    # data[i] have now ["full"] and ["running"]

doing pathensemble 0
doing pathensemble 1
Weights of the different paths:
wRMR = 0
wRML = 2149
wLMR = 1504
wLML = 39605
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.036585662506993605
pLML = 0.9634143374930064
Local crossing probabilities:
p2R = 0.03476813537380369
p2L = 0.9652318646261963
doing pathensemble 2
Weights of the different paths:
wRMR = 2037
wRML = 481
wLMR = 571
wLML = 13588
Local crossing probabilities:
pRMR = 0.8089753772835584
pRML = 0.19102462271644163
pLMR = 0.040327706758951906
pLML = 0.9596722932410481
Local crossing probabilities:
p2R = 0.15638304251364155
p2L = 0.8436169574863585
doing pathensemble 3
Weights of the different paths:
wRMR = 5803
wRML = 1781
wLMR = 1966
wLML = 22810
Local crossing probabilities:
pRMR = 0.7651635021097046
pRML = 0.23483649789029537
pLMR = 0.07935098482402325
pLML = 0.9206490151759767
Local crossing probabilities:
p2R = 0.24008034610630408
p2L = 0.7599196538936959
doing pathensemble 4
Weights of the different paths:
wRM

### Generate pathlength distribution figures, as in PyRETIS reports.

In [30]:
for i, pe in enumerate(pathensembles):
    upe = pe.unify_pe()
    # Pathlength distribution
    data[i]["pathlengths"] = pathlength_distr(upe)  # these might be used later or not! TODO
        
#=======================================
# make figures
makefigs = True 
if makefigs:
    for i, pe in enumerate(pathensembles):     
        if i == 0:
            continue
        # Cross distances distribution
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
        fig,ax = plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution      
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)

Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True


KeyboardInterrupt: 

### Compute Pcross using in-house functions

In [31]:
# Global crossing probabilities (no error analysis)  
psfull = []
for i in range(1, len(pathensembles)):   # do not use the 0- ensemble
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})

Pminfull, Pplusfull, Pcrossfull = get_global_probs_from_dict(psfull)

In [49]:
# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(Pcrossfull, "o", c = "r")
ax.errorbar([i for i in range(len(Pcrossfull))], Pcrossfull, fmt="-o", c = "b", ecolor="r", capsize=6)


ax.set_xlabel("intf")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
ax.set_xticks(np.arange(len(interfaces)))
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")

print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
print("which is the case!")
print(Pcrossfull)
print([Pcrossfull[i]/Pcrossfull[i-1] for i in range(1,len(Pcrossfull))])
print("Here, the load immediately disappeared. For a simulation where this is")
print("not the case, the above code should be adapted a little bit.")

This should be the same as the repptis_report.pdf value: 0.0001335109597375244
which is the case!
[1.0, 0.036585662506993605, 0.0014754158691640193, 0.00045872892420405605, 0.00025380073052279486, 0.0001648882630423318, 0.00014568491255856423, 0.0001373779250610477, 0.00013493581615718747, 0.00013439213312617257, 0.000133904060831669, 0.0001335109597375244]
[0.036585662506993605, 0.040327706758951906, 0.31091499948687346, 0.5532695174239697, 0.6496760773804098, 0.8835371897947795, 0.9429797681062053, 0.9822234256138677, 0.9959708026638268, 0.9963682971380077, 0.9970643078954957]
Here, the load immediately disappeared. For a simulation where this is
not the case, the above code should be adapted a little bit.


In [33]:
# TODO DONT INCLUDE??

# Construct lists of the local probs

# Or we can use the get_global_probs_from_local function, using lists of the local probs
# These do not use the 0- ensemble
pmps = [data[i]["full"]["LMR"] for i in range(1,len(pathensembles))]
pmms = [data[i]["full"][
    "LML"] for i in range(1,len(pathensembles))]
ppps = [data[i]["full"]["RMR"] for i in range(1,len(pathensembles))]
ppms = [data[i]["full"]["RML"] for i in range(1,len(pathensembles))]
a,b,c = get_global_probs_from_local(pmps, pmms, ppps, ppms)
print("This should be the same as the repptis_report.pdf value:", c[-1])
print(c)
print([c[i]/c[i-1] for i in range(1,len(c))])

This should be the same as the repptis_report.pdf value: 0.0001335109597375244
[1.0, 0.036585662506993605, 0.0014754158691640193, 0.00045872892420405605, 0.00025380073052279486, 0.0001648882630423318, 0.00014568491255856423, 0.0001373779250610477, 0.00013493581615718747, 0.00013439213312617257, 0.000133904060831669, 0.0001335109597375244]
[0.036585662506993605, 0.040327706758951906, 0.31091499948687346, 0.5532695174239697, 0.6496760773804098, 0.8835371897947795, 0.9429797681062053, 0.9822234256138677, 0.9959708026638268, 0.9963682971380077, 0.9970643078954957]


## 4. Analysis using the MSM

### Construct transition matrix M

In [34]:
print(interfaces)
N = len(interfaces)
NS = 4*N-5
print("N", N)
# print("len pmms", len(pmms)) # TODO INCLUDE?
print("NS", NS)

labels1, labels2 = create_labels_states(N)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.5, 8.0, 10.0, 12.0, 14.0, 16.0, 19.0]
N 12
NS 43


In [35]:
print("mm", pmms)
print("mp", pmps)
print("pm", ppms)
print("pp", ppps)
print("sum", np.array(pmms)+np.array(pmps))
print("sum", np.array(ppms)+np.array(ppps))
if N > 3:  
    M = construct_M(pmms, pmps, ppms, ppps, N)
elif N == 3:
    M = construct_M_N3(pmms, pmps, ppms, ppps, N)
else:
    raise ValueError("The amount of interfaces needs to be 3 at least!")

mm [0.9634143374930064, 0.9596722932410481, 0.9206490151759767, 0.8212281000170097, 0.944436609787054, 0.7982261640798226, 0.8482965009208103, 0.5104129897635016, 0.545131958305611, 0.7721486100285788, 0.6520530367835757]
mp [0.036585662506993605, 0.040327706758951906, 0.07935098482402325, 0.1787718999829903, 0.055563390212945984, 0.2017738359201774, 0.15170349907918967, 0.4895870102364984, 0.454868041694389, 0.22785138997142115, 0.3479469632164243]
pm [1.0, 0.19102462271644163, 0.23483649789029537, 0.0583185955225812, 0.08982647158897585, 0.0741162129215766, 0.25826446280991733, 0.09692355988070947, 0.1455205411648243, 0.3340473640366536, 0.4607300365956648]
pp [0.0, 0.8089753772835584, 0.7651635021097046, 0.9416814044774188, 0.9101735284110242, 0.9258837870784234, 0.7417355371900827, 0.9030764401192906, 0.8544794588351757, 0.6659526359633464, 0.5392699634043352]
sum [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
sum [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [36]:
# We can print the transition matrix M and check that all rows sum to 1.
print("M")
print("shape", M.shape)
print("sum prob in rows", np.sum(M,axis=1))
print(M)

M
shape (43, 43)
sum prob in rows [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.         0.96341434 0.03658566 ... 0.         0.         0.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]]


### Look at this Markov model
*INCLUDE?*

In [37]:
#import numpy.linalg
vals, vecs = np.linalg.eig(M)
print(vals)
vals, vecs = np.linalg.eig(M.T)
print(vals)
pprint(M)

[-9.99871551e-01+0.00000000e+00j -9.78096982e-01+0.00000000e+00j
 -9.56950716e-01+0.00000000e+00j -9.27231768e-01+0.00000000e+00j
 -8.88544157e-01+0.00000000e+00j -8.07877897e-01+0.00000000e+00j
 -7.38502810e-01+0.00000000e+00j -6.88749732e-01+0.00000000e+00j
 -5.56267715e-01+0.00000000e+00j -4.84066806e-01+0.00000000e+00j
  3.42493313e-14+1.86651192e-01j  3.42493313e-14-1.86651192e-01j
  4.84066783e-01+0.00000000e+00j  5.56267948e-01+0.00000000e+00j
  1.00000000e+00+0.00000000e+00j  9.77414399e-01+0.00000000e+00j
  9.58035571e-01+0.00000000e+00j  9.26411830e-01+0.00000000e+00j
  8.88886766e-01+0.00000000e+00j  6.88740556e-01+0.00000000e+00j
  7.38535090e-01+0.00000000e+00j  8.07801189e-01+0.00000000e+00j
 -9.51771231e-09+0.00000000e+00j  9.51771236e-09+0.00000000e+00j
 -9.20480039e-17+1.30318048e-16j -9.20480039e-17-1.30318048e-16j
  6.07646083e-18+1.36595519e-16j  6.07646083e-18-1.36595519e-16j
 -2.34505938e-17+7.28796888e-17j -2.34505938e-17-7.28796888e-17j
 -6.77671651e-17+0.000000

In [38]:
print("what if chain propagates")
print("A[0,:]")
# check stationary behavior
A = M
for n in range(10):
    A = np.dot(A,M)
    #print(A)
    print(A[0,:])
    print(np.sum(A[0,:]))  # is 1 indeed

what if chain propagates
A[0,:]
[0.96341434 0.         0.         0.         0.03511025 0.00147542
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
1.0
[0.00000000e+00 9.28167186e-01 3.52471518e-02 3.51102466e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.35834017e-03 1.17075702e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e

### Pcross with MSM

In [39]:
# Inspect Z and Y vectors

z1, z2, y1, y2 = global_pcross_msm(M)
print("Z")
print_vector(z1, labels1)
print_vector(z2, labels2)
print("Y")
print_vector(y1, labels1)
print_vector(y2, labels2)
print("\nGlobal crossing probability: ", y1[0][0])

Z
state 0-     : 0
state B      : 1
state 0+- LML: 0.0
state 0+- LMR: 0.003649269975964023
state 0+- RML: 0.0
state 1+- LML: 0.0
state 1+- LMR: 0.09049039157561226
state 1+- RML: 0.0
state 1+- RMR: 0.09049039157561226
state 2+- LML: 0.07320449866541787
state 2+- LMR: 0.2910454359711026
state 2+- RML: 0.07320449866541785
state 2+- RMR: 0.2910454359711026
state 3+- LML: 0.2398884331570962
state 3+- LMR: 0.5260463965667476
state 3+- RML: 0.2398884331570962
state 3+- RMR: 0.5260463965667476
state 4+- LML: 0.5093580660430945
state 4+- LMR: 0.8097056593000083
state 4+- RML: 0.5093580660430945
state 4+- RMR: 0.8097056593000083
state 5+- LML: 0.7827264947474988
state 5+- LMR: 0.9164364201670786
state 5+- RML: 0.7827264947474989
state 5+- RMR: 0.9164364201670786
state 6+- LML: 0.9065263468649529
state 6+- LMR: 0.9718516252024854
state 6+- RML: 0.9065263468649529
state 6+- RMR: 0.9718516252024854
state 7+- LML: 0.9549804272847342
state 7+- LMR: 0.989440487631518
state 7+- RML: 0.9549804272847342

### Pathlength analysis

In [40]:
# Setting path ensemble properties
#==================================
for i,fol in enumerate(folders):
    print(i)
    print("Calculating path lengths.")
    set_tau_distrib(pathensembles[i])
    print("Done.")

    if True:
        print("Calculating first hitting lengths to middle interface")
        set_tau_first_hit_M_distrib(pathensembles[i])
        print("Done.")

0
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
1
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
2
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
3
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
4
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
5
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
6
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
7
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
8
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
9
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
10
Calculating path lengths.
Done.
Calculating first hitting

In [ ]:
# Additional information
#==================================
# Average path lengths per ensemble for each path type
print(indir[-20:])
pathtypes = ("LML", "LMR", "RMR", "RML", "LM*", "*M*", "***", "RM*", "L**", "**R", "R**")

print("=" * 80)
print("AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE")
print("=" * 80)

for i, pe in enumerate(pathensembles):
    print(f"\nEnsemble {i} ({pe.name}):")
    print("-" * 50)
    
    # Get accepted paths only
    accepted_mask = np.isin(pe.flags, ACCFLAGS)
    total_accepted_count = np.sum(accepted_mask)
    
    if total_accepted_count > 0:
        # Calculate weighted average for all accepted paths
        accepted_lengths = pe.lengths[accepted_mask]
        accepted_weights = pe.weights[accepted_mask]
        total_weighted_avg = np.average(accepted_lengths, weights=accepted_weights)
        
        print(f"  All accepted paths: {total_weighted_avg:8.2f} (n={total_accepted_count:4d}, weighted)")
        print("-" * 30)
    
    for ptype in pathtypes:
        mask = (pe.lmrs == ptype) & accepted_mask
        if np.any(mask):
            lengths = pe.lengths[mask]
            weights = pe.weights[mask]
            weighted_avg = np.average(lengths, weights=weights)
            count = np.sum(mask)
            print(f"  {ptype:4s}: {weighted_avg:8.2f} (n={count:4d}, weighted)")
        else:
            print(f"  {ptype:4s}: {0:8.2f} (n={0:4d}, weighted)")

s_data_sc10_12intfs/
AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE

Ensemble 0 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000):
--------------------------------------------------
  All accepted paths:    40.64 (n=9037, weighted)
------------------------------
  LML :     0.00 (n=   0, weighted)
  LMR :     0.00 (n=   0, weighted)
  RMR :    40.64 (n=9037, weighted)
  RML :     0.00 (n=   0, weighted)
  LM* :     0.00 (n=   0, weighted)
  *M* :     0.00 (n=   0, weighted)
  *** :     0.00 (n=   0, weighted)
  RM* :     0.00 (n=   0, weighted)
  L** :     0.00 (n=   0, weighted)
  **R :     0.00 (n=   0, weighted)
  R** :     0.00 (n=   0, weighted)

Ensemble 1 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001):
--------------------------------------------------
  All accepted paths:    51.54 (n=9454, weighted)
------------------------------
  LML :    36.20 (n=8980, weighted

: 

In [41]:
# Compute taus for pathlength analysis
tau_mm, tau_mp, tau_pm, tau_pp = collect_tau(pathensembles)
tau1_mm, tau1_mp, tau1_pm, tau1_pp = collect_tau1(pathensembles)
tau2_mm, tau2_mp, tau2_pm, tau2_pp = collect_tau2(pathensembles)
taum_mm, taum_mp, taum_pm, taum_pp = collect_taum(pathensembles)

Collect tau
ensemble 0 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000
ensemble 1 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001
ensemble 2 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/002
ensemble 3 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/003
ensemble 4 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/004
ensemble 5 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/005
ensemble 6 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/006
ensemble 7 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/007
ensemble 8 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTI

In [42]:
# Look at computed taus
print("tau")
print_all_tau(pathensembles, tau_mm, tau_mp, tau_pm, tau_pp)
print("\ntau1")
print_all_tau(pathensembles, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
print("\ntaum")
print_all_tau(pathensembles, taum_mm, taum_mp, taum_pm, taum_pp)
print("\ntau2")
print_all_tau(pathensembles, tau2_mm, tau2_mp, tau2_pm, tau2_pp)

tau
Index Name            mm           mp           pm           pp
-----------------------------------------------------
0     000            nan          nan          nan         39.8
1     001           33.0        249.7        177.7          nan
2     002         4327.9       3938.9      10217.9       2290.2
3     003          755.8       1650.1       1633.6       1330.6
4     004         1187.4       2894.9       2515.8       1030.1
5     005         1835.2       4506.5       3706.1       4655.1
6     006         3364.5      11320.2       7152.9       2479.4
7     007         9177.9      22148.2      25658.3       8269.3
8     008         4368.0       5068.7       4324.9       3463.2
9     009         2550.9       3163.6       3104.3       3768.6
10    010         2770.8       2304.8       2609.2       1595.1
11    011         1890.5       2109.4       1932.5       2059.9

tau1
Index Name            mm           mp           pm           pp
----------------------------------------

In [43]:
# TODO include prints?
tau  = construct_tau_vector(N, NS, tau_mm, tau_mp, tau_pm, tau_pp)
tau1 = construct_tau_vector(N, NS, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
taum = construct_tau_vector(N, NS, taum_mm, taum_mp, taum_pm, taum_pp)
tau2 = construct_tau_vector(N, NS, tau2_mm, tau2_mp, tau2_pm, tau2_pp)
tau_m = tau-tau1-tau2  # yes, this is the same thing as taum

print("tau")
print(tau)
print("\n")
print("tau1")
print(tau1)
print("taum")
print(taum)
print("tau2")
print(tau2)

print("\n")
print("tau = tau1+taum+tau2 => difference is", np.sum((tau-tau1-taum-tau2)**2))

tau
[   39.82187826    32.99224845   249.65824468   177.70590973
  4327.87150427  3938.87915937 10217.92723493  2290.15414826
   755.8167032   1650.11597152  1633.55362156  1330.60641048
  1187.42916321  2894.89533777  2515.76257396  1030.14222894
  1835.19882037  4506.51395939  3706.09090909  4655.10130841
  3364.54320988 11320.19169719  7152.91666667  2479.39366278
  9177.93568521 22148.23368741 25658.3472      8269.32089136
  4367.95573997  5068.72819034  4324.86315789  3463.18354045
  2550.90439382  3163.57386641  3104.31179138  3768.61604557
  2770.81280843  2304.82022045  2609.20662629  1595.12794853
  1890.52891658  2109.42982995     0.        ]


tau1
[   0.            0.            0.          177.70590973  176.19333235
  172.9614711   161.44282744  215.63966618  149.64739149  122.474059
  128.69960696  153.71531966  164.99026512  189.69267364  136.94970414
  194.47185195  217.62714648  156.19923858  384.10984848 1007.75607477
  585.51141975  515.73992674  365.49561404  424.78

## 5. Flux calculation

### Collect tau for [0+]

In [44]:
# Construct g and h vectors
g1, g2, h1, h2 = mfpt_to_first_last_state(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2)) #, doprint=True)
print("G")
print_vector(g1, labels1)
print_vector(g2, labels2)
print("H")
print_vector(h1, labels1)
print_vector(h2, labels2)
print("\ntau [0+]: ", h1[0])

[ 0 42] g1:  [[0.]
 [0.]] D [[1. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]] tp [[   32.99224845]
 [  249.65824468]
 [    0.        ]
 [ 4151.67817192]
 [ 3765.91768827]
 [10056.48440748]
 [ 2074.51448208]
 [  606.16931171]
 [ 1527.64191251]
 [ 1504.8540146 ]
 [ 1176.89109082]
 [ 1022.43889809]
 [ 2705.20266413]
 [ 2378.81286982]
 [  835.67037699]
 [ 1617.57167388]
 [ 4350.31472081]
 [ 3321.98106061]
 [ 3647.34523364]
 [ 2779.03179012]
 [10804.45177045]
 [ 6787.42105263]
 [ 2054.60730273]
 [ 8611.86485753]
 [21769.15174507]
 [24529.2224    ]
 [ 7072.75821727]
 [ 3223.2164592 ]
 [ 4189.72314348]
 [ 3934.27854251]
 [ 2881.42000521]
 [ 1917.83157038]
 [ 2600.99317406]
 [ 26

### The flux

In [45]:
flux = 1/(tau[0]+h1[0][0])
dt = 0.002 # Change if needed
sc = 10
flux /= (dt*sc)
# print(flux/(dt*sc), "1/time")
print(flux, "1/ps")

0.15937138415510635 1/ps


## 6. The rate constant
We can compute an accurate rate constant using only our MSM.

In [46]:
# rate constant = flux * Pcross

print("The rate constant k is: ", flux*y1[0][0], "1/ps")

The rate constant k is:  2.127782645324594e-05 1/ps


## 7. Direct rate computation via $\tau_{\mathcal{A},1}$ 

In [47]:
# Construct g and h vectors
absor = np.array([NS - 1])
kept = np.array([i for i in range(NS) if i not in absor])

g1, g2, h1, h2 = mfpt_to_absorbing_states(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2), absor, kept, remove_initial_m=False, doprint=True)
print("G")
print_vector(g1, labels1[-1])
print_vector(g2, [labels1[0]] + labels2)
print("H")
print_vector(h1, labels1[-1])
print_vector(h2, [labels1[0]] + labels2)
print("interesting")
print(h2[0])
mfpt = h2[0][0]  # tau_A,1

[42] g1:  [[0.]] D [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]] tp [[   39.82187826]
 [   32.99224845]
 [  249.65824468]
 [    0.        ]
 [ 4151.67817192]
 [ 3765.91768827]
 [10056.48440748]
 [ 2074.51448208]
 [  606.16931171]
 [ 1527.64191251]
 [ 1504.8540146 ]
 [ 1176.89109082]
 [ 1022.43889809]
 [ 2705.20266413]
 [ 2378.81286982]
 [  835.67037699]
 [ 1617.57167388]
 [ 4350.31472081]
 [ 3321.98106061]
 [ 3647.34523364]
 [ 2779.03179012]
 [10804.45177045]
 [ 6787.42105263]
 [ 2054.60730273]
 [ 8611.86485753]
 [21769.15174507]
 [24529.2224    ]
 [ 7072.75821727]
 [ 3223.2164592 ]
 [ 4189.72314348]
 [ 3934.27854251]
 [ 2881.42000521]
 [ 1917.83157038]
 [ 2600.99317406]
 [ 2676.42517007]
 [ 3093.36763854]
 [ 2181.52030058]
 [ 1890.20334474]
 [ 2295.09690061]
 [ 1258.60543245]
 [ 15

In [48]:
k_flux_pcross = flux * y1[0][0]
k_mfpt = 1 / (mfpt*dt*sc)

print(f"Rate constant from P_cross × flux: {k_flux_pcross:.10e} [1/ps]")
print(f"Rate constant from MFPT:           {k_mfpt:.10e} [1/ps]")
print(f"Relative difference:               {abs(k_flux_pcross - k_mfpt)/k_mfpt*100:.2f}%")


Rate constant from P_cross × flux: 2.1277826453e-05 [1/ps]
Rate constant from MFPT:           2.1277826453e-05 [1/ps]
Relative difference:               0.00%
